In [1]:

import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras
from keras.layers import *

print(tf.__version__)


C:\Users\Kiko-PC\Anaconda3\envs\tensorflow\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)


1.12.0


Using TensorFlow backend.


In [2]:


def ConvBnAct(x, n_filters=64, kernel=(2,2), strides=(1,1), padding='valid', activation=tf.nn.relu):
  
  
        filters = n_filters
        kernel = kernel
        strides = strides
        activation = tf.nn.relu
        
        #print('padding', padding)
        conv_ =  Conv2D(filters=filters,
                      kernel_size = kernel,
                      strides = strides, 
                      padding=padding)

        batch_norm = BatchNormalization()

        activation = Activation(activation)
    

        x = conv_(x) 
        x = batch_norm(x)

        x = activation(x)

        return x

In [3]:
norm = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)
t = np.random.randn(10, 256, 256, 64)
block = ConvBnAct(norm)



In [4]:
def ConvAct( x, n_filters, kernel=(1,1), activation = tf.nn.relu, pooling=False):
  


      poolingLayer = AveragePooling2D(pool_size=(1,1), padding='same')
      convLayer = Conv2D(filters = n_filters,
                         kernel_size = kernel,
                         strides=1)

      activation = Activation(activation)

      if pooling:
        x = poolingLayer(x)

      x = convLayer(x)
      x = activation(x)

      return x

In [5]:

norm = tf.random_normal([10, 127, 127, 64], mean=-1, stddev=4)

block = ConvAct(norm, n_filters=64)
print(block)
print(block.shape)

Tensor("activation_2/Relu:0", shape=(10, 127, 127, 64), dtype=float32)
(10, 127, 127, 64)


In [6]:
def AttentionRefinmentModule(inputs, n_filters):
  
    filters = n_filters
    
    poolingLayer = AveragePooling2D(pool_size = (1,1), padding='same')
    
    x = poolingLayer(inputs)
    
    x = ConvBnAct(x, kernel = (1,1), n_filters = filters, activation = tf.nn.sigmoid)
    
    print(x.shape)
    print(inputs.shape)
    
    return multiply([inputs,x])

In [7]:
norm = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)

block = AttentionRefinmentModule(norm, n_filters=64)

print(block.shape)

(10, 256, 256, 64)
(10, 256, 256, 64)
(10, 256, 256, 64)


In [8]:

def FeatureFusionModule(input_f, input_s,n_filters):
#     self.conv_bn_act = ConvBnAct(n_filters=n_filters, kernel=(3, 3), padding='same')
#     self.conv_act1 = ConvAct(n_filters=n_filters, pooling=True)
#     self.conv_act2 = ConvAct(n_filters=n_filters, pooling=False, activation = tf.nn.sigmoid)
    activation = tf.nn.sigmoid
    
       
    concate = Concatenate(axis=-1)([input_f, input_s])
    
    branch0 =  ConvBnAct(concate, n_filters=n_filters, kernel=(3, 3), padding='same') 
    branch_1  = ConvAct(branch0, n_filters=n_filters, pooling=True, activation = tf.nn.relu)
    branch_1 = ConvAct(branch_1, n_filters=n_filters, pooling=False, activation = tf.nn.sigmoid)
    
    x = multiply([branch0, branch_1])
    return  Add()([branch0, x])


In [9]:
norm1 = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)
norm2 = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)

block = FeatureFusionModule(norm1, norm2, n_filters=64)

print(block.shape)

(10, 256, 256, 64)


In [16]:


def ContextPath(layer_13, layer_14):
    
    globalmax = GlobalAveragePooling2D()
    
    net_4 = AttentionRefinmentModule(layer_13, n_filters=1024)
    net_5 = AttentionRefinmentModule(layer_14, n_filters=2048)
    
    global_channels = globalmax(net_5)
    net_5_scaled = multiply([global_channels, net_5])
    
    net_4 = UpSampling2D(size=(4,4),  interpolation='bilinear')(net_4)
    net_5_scaled =  UpSampling2D(size=(4,4),  interpolation='bilinear')(net_5_scaled)

    
    cnc = Concatenate(axis=-1)([net_4, net_5_scaled])
 
    return cnc

In [17]:

layer_13 = tf.random_normal([10, 7, 7, 1024], mean=-1, stddev=4)
layer_14 = tf.random_normal([10, 7, 7, 2048], mean=-1, stddev=4)

block = ContextPath(layer_13,layer_14)

print(block.shape)

(10, 7, 7, 1024)
(10, 7, 7, 1024)
(10, 7, 7, 2048)
(10, 7, 7, 2048)
(10, 28, 28, 3072)


In [18]:
# (10, 7, 7, 512)
# (10, 7, 7, 512)
# (10, 7, 7, 2048)
# (10, 7, 7, 2048)
# (10, 28, 28, 4608)

In [45]:
def FinalModel(x, layer_13, layer_14):

    
    #upsample = Upsampling(size=(8, 8))
   # self.feature_fusion = FeatureFusionModule(32)

    #spatial path
    x = ConvBnAct(x, 32, strides=2)
    x = ConvBnAct(x, 64, strides=2)
    x = ConvBnAct(x, 156, strides=2)
    
    #context path
    cp = ContextPath(layer_13, layer_14)
    
   
    print('fusion')
    fusion = FeatureFusionModule(cp, x, 32)
    print('fusion')
    ans = UpSampling2D(size=(8,8),  interpolation='bilinear')(fusion)
    
    
    return ans


In [46]:
# for layer in model.layers[:5]:
#     layer.trainable = False
input_layer = tf.random_normal([10, 224, 224, 3], mean=-1, stddev=4)

layer_13 = tf.random_normal([10, 7, 7, 1024], mean=-1, stddev=4)
layer_14 = tf.random_normal([10, 7, 7, 2048], mean=-1, stddev=4)

block = FinalModel(input_layer, layer_13, layer_14)

print(block.shape)

(10, 7, 7, 1024)
(10, 7, 7, 1024)
(10, 7, 7, 2048)
(10, 7, 7, 2048)
fusion
fusion
(10, 224, 224, 32)


In [47]:
def load_image(path):
    image = cv2.cvtColor(cv2.imread(path,-1), cv2.COLOR_BGR2RGB)
    return image



In [48]:
import os 
import numpy as np 
from keras.preprocessing import image
import cv2

PATH = os.getcwd()
train_path = PATH+'\data\\train\\'
trainy_path = PATH+'\data\\train_labels\\'

val_path = PATH+'\data\\val\\'
valy_path = PATH+'\data\\val_labels\\'

print(train_path)

train_batch = os.listdir(train_path)
trainy_batch = os.listdir(trainy_path)

val_batch = os.listdir(val_path)
valy_batch = os.listdir(valy_path)

X_train = []
y_train = []

X_val = []
y_val = []

# if data are in form of images
for sample in train_batch:
    img_path = train_path+sample
    x = load_image(img_path)
    x = cv2.resize(x, dsize=(608, 608))
    
    X_train.append(x)


for sample in trainy_batch:
    img_path = trainy_path+sample
    x = load_image(img_path)
    x = cv2.resize(x, dsize=(608, 608))


    y_train.append(x)
    


    
# if data are in form of images
for sample in val_batch:
    img_path = val_path+sample
    x = load_image(img_path)
    x = cv2.resize(x, dsize=(608, 608))

    X_val.append(x)


for sample in valy_batch:
    img_path = valy_path+sample
    x = load_image(img_path)
    x = cv2.resize(x, dsize=(608, 608))


    y_val.append(x)



C:\Users\Kiko-PC\Documents\Image-Segmentation\data\train\


In [49]:
    # X_train = np.array(X_train)
    # y_train = np.array(y_train)

In [50]:
# X_val = np.array(X_val)
# y_val = np.array(y_val)

In [51]:
#print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)#

In [52]:

def get_label_info(csv_path):
    """
    Retrieve the class names and label values for the selected dataset.
    Must be in CSV format!

    # Arguments
        csv_path: The file path of the class dictionairy
        
    # Returns
        Two lists: one for the class names and the other for the label values
    """
    filename, file_extension = os.path.splitext(csv_path)
    if not file_extension == ".csv":
        return ValueError("File is not a CSV!")

    class_names = []
    label_values = []
    with open(csv_path, 'r') as csvfile:
        file_reader = csv.reader(csvfile, delimiter=',')
        header = next(file_reader)
        for row in file_reader:
            class_names.append(row[0])
            label_values.append([int(row[1]), int(row[2]), int(row[3])])
        # print(class_dict)
    return class_names, label_values

def one_hot_it(label, label_values):
    """
    Convert a segmentation image label array to one-hot format
    by replacing each pixel value with a vector of length num_classes

    # Arguments
        label: The 2D array segmentation image label
        label_values
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of num_classes
    """
   
    semantic_map = []
    for colour in label_values:
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)

    return semantic_map
    
def reverse_one_hot(image):
    """
    Transform a 2D array in one-hot format (depth is num_classes),
    to a 2D array with only 1 channel, where each pixel value is
    the classified class key.

    # Arguments
        image: The one-hot format image 
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of 1, where each pixel value is the classified 
        class key.
    """
    # w = image.shape[0]
    # h = image.shape[1]
    # x = np.zeros([w,h,1])

    # for i in range(0, w):
    #     for j in range(0, h):
    #         index, value = max(enumerate(image[i, j, :]), key=operator.itemgetter(1))
    #         x[i, j] = index

    x = np.argmax(image, axis = -1)
    return x


def colour_code_segmentation(image, label_values):
    """
    Given a 1-channel array of class keys, colour code the segmentation results.

    # Arguments
        image: single channel array where each value represents the class key.
        label_values
        
    # Returns
        Colour coded image for segmentation visualization
    """

    # w = image.shape[0]
    # h = image.shape[1]
    # x = np.zeros([w,h,3])
    # colour_codes = label_values
    # for i in range(0, w):
    #     for j in range(0, h):
    #         x[i, j, :] = colour_codes[int(image[i, j])]
    
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]

    return x


In [53]:
import csv
class_names_list, label_values =get_label_info("data/class_dict.csv")
class_names_string = ""
for class_name in class_names_list:
    if not class_name == class_names_list[-1]:
        class_names_string = class_names_string + class_name + ", "
    else:
        class_names_string = class_names_string + class_name

num_classes = len(label_values)

In [54]:
print(num_classes)

32


In [55]:
# y_train = np.float32(one_hot_it(y_train[:200], label_values=label_values))
from sklearn.utils import shuffle

a =np.array([[1,2,3],[2,3,4]])
b = np.array([[4,5,6],[7,8,9]])

shuffle(a, b)

[array([[1, 2, 3],
        [2, 3, 4]]), array([[4, 5, 6],
        [7, 8, 9]])]

In [56]:
#print(y_train.shape)

In [57]:
img = np.random.randn(244,244,3)
s = np.float32(one_hot_it(label=img, label_values=label_values))
print(s.shape)

(244, 244, 32)


In [58]:
from sklearn.utils import shuffle


def train_generator(samplesX, samplesY, label_values, batch_size=1, is_training=True):
    """
    Lazy batch train/validation generator for memory efficiency
    """
    num_samples = len(samplesX)

    
    samplesX, samplesY = shuffle(samplesX, samplesY)


    while 1: # Loop forever so the generator never terminates
        #shuffle the samples once the whole data is processed into batches
        #split data into batches
        for offset in range(0, num_samples, batch_size):
            X_train = samplesX[offset:offset+batch_size]
            y_train = samplesY[offset:offset+batch_size]
            
                # preprocessing if required
            X_f = []
            y_f = []
            for x,y in zip(X_train, y_train):


                y = np.float32(one_hot_it(y, label_values=label_values))
                X_f.append(x)
                y_f.append(y)
#             print('--')
#             print(X_train.shape)
#             print(y_train.shape)
            X_f = np.float32(X_f)
            y_f = np.float32(y_f)
            yield ([X_f, X_f], y_f)

In [59]:
from keras.models import Model

In [60]:
from keras.applications import *
#from keras.applications import preprocess_input


Xception_model  = Xception(weights='imagenet',input_shape= (608,608,3), include_top=False)

# 16x Down

inputs = Input(shape=(608,608,3))
x = Lambda(lambda image: preprocess_input(image))(inputs)

    
#spatial path



In [61]:

layer_13 = Xception_model.get_layer('block13_pool').output
layer_14 = Xception_model.output

ans = FinalModel(x, layer_13, layer_14)



(?, 19, 19, 1024)
(?, 19, 19, 1024)
(?, 19, 19, 2048)
(?, 19, 19, 2048)
fusion
fusion


In [62]:

# for layer in (xception.layers[:100]):
#     layer.trainable=False


In [63]:
print(x.shape)

(?, 608, 608, 3)


In [64]:
model = Model(inputs = [inputs, Xception_model.input], output = [ans])

C:\Users\Kiko-PC\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [65]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 303, 303, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 303, 303, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 303, 303, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [66]:
# X_train = tf.convert_to_tensor(
#     X_train[:10],

# )
# y_train = tf.convert_to_tensor(
#     y_train[:10],

# )

In [70]:
BATCH_SIZE = 1
EPOCHS = 1000

from keras.callbacks import ModelCheckpoint



#model.compile(optimizer=tf.train.RMSPropOptimizer(learning_rate=0.0001, decay=0.99), 
def categorical_crossentropy(y_true, y_pred):
    return K.categorical_crossentropy(y_true, y_pred, from_logits=True)

model.compile(optimizer=tf.train.RMSPropOptimizer(learning_rate=0.001), 
              loss=categorical_crossentropy,
              metrics=['accuracy'],
              )


#Get model training checkpoints
checkpoint = ModelCheckpoint('checkpoints/model-test-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 mode='auto')


#model = load_model('checkpoints/model.h5')
#generator = train_generator(X_train, y_train, label_values)

                                   

In [71]:
generator = train_generator(X_train, y_train, label_values)
validation_generator = train_generator(X_val, y_val, label_values)


In [72]:
history_object = model.fit_generator(generator,
                                     len(X_train)//BATCH_SIZE,
                                     epochs=EPOCHS, 
                                     validation_data=validation_generator,
                                     validation_steps = len(X_val)//BATCH_SIZE,
                                     callbacks=[checkpoint])

Epoch 1/1000
421/421 [==============================] - 184s 437ms/step - loss: 1.3825 - acc: 0.7053 - val_loss: 0.8433 - val_acc: 0.7816


C:\Users\Kiko-PC\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\saving.py:118: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


Epoch 2/1000
421/421 [==============================] - 173s 411ms/step - loss: 0.6212 - acc: 0.8345 - val_loss: 1.5116 - val_acc: 0.7812
Epoch 3/1000
421/421 [==============================] - 176s 417ms/step - loss: 0.4360 - acc: 0.8645 - val_loss: 0.5689 - val_acc: 0.8287
Epoch 4/1000
 36/421 [=>............................] - ETA: 1:44 - loss: 0.3456 - acc: 0.8763

KeyboardInterrupt: 